**this is the pre-processing pipeline for selective help**

1) download the data from Lookit <br> 2) upload the local raw JSON file



In [ ]:
import json
import csv
import os
import pandas as pd

from google.colab import files
uploaded = files.upload()

Saving all_responses_identifiable.json to all_responses_identifiable.json


3) preprocessing steps

In [ ]:
# load the JSON data
with open('all_responses_identifiable.json', 'r') as file:
    data = json.load(file)

# init spreadsheet rows
rows = []

# init list of participants to check for duplicates
acceptedChildHashedIds = []

def get_study_and_counterbalance(exp_data):
    """Extract study version (1A/1B) and counterbalance (1-8) from study video"""
    study_video = None
    if '6-study-video' in exp_data:
        study_video = exp_data['6-study-video'].get('videoShown', '')

    if not study_video:
        return "NA", "NA"

    if "CB" in study_video:
        video_name = study_video.split('/')[-1].split('.')[0]
        try:
            cb_num = int(video_name[2])
            if '_1A' in video_name:
                return '1A', cb_num
            elif '_1B' in video_name:
                return '1B', cb_num
        except:
            pass

    return "NA", "NA"

def get_selection_response(exp_data):
    """Extract whether participant selected Casey (left) or Alex (right)"""
    if '7-test-question' in exp_data:
        selected_image = exp_data['7-test-question'].get('selectedImage', '')
        if selected_image == "left-choice":
            return "Casey"
        elif selected_image == "right-choice":
            return "Alex"
    return "NA"

# get each participant's data
for response_entry in data:
    try:
        childData = response_entry['child']
        hashedId = childData['hashed_id']
        responseData = response_entry['response']
        expData = response_entry['exp_data']

        # make sure that it's not a duplicate, then proceed to get data
        if hashedId not in acceptedChildHashedIds:
            study_version, counterbalance = get_study_and_counterbalance(expData)
            selection = get_selection_response(expData)

            row = [
                hashedId,
                response_entry['participant']['hashed_id'],
                responseData['date_created'],
                round(float(childData['age_in_days']) / 365.25, 2),
                childData['age_in_days'],
                childData['gender'],
                study_version,
                counterbalance,
                selection,
                responseData['completed'],
                responseData.get('is_preview', False)
            ]

            rows.append(row)
            acceptedChildHashedIds.append(hashedId)

    except Exception as e:
        continue

# create a df
columns = [
    'child_hashed_id',
    'parent_hashed_id',
    'response_date_created',
    'age_years',
    'child_age',
    'child_gender',
    'study_version',
    'counterbalance',
    'selection',
    'completed',
    'is_preview'
]

df = pd.DataFrame(rows, columns=columns)

4) check to make sure it all looks good before downloading

In [ ]:
df

,child_hashed_id,parent_hashed_id,response_date_created,age_years,child_age,child_gender,study_version,counterbalance,selection,completed,is_preview
0,PMHcSH,YFX63T,2025-01-14 04:53:50.536206+00:00,4.92,1798,m,1B,5,Casey,True,True


5) save to csv!

In [ ]:
# save to CSV
csv_filename = 'participant_data.csv'
df.to_csv(csv_filename, index=False)
files.download(csv_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>